In [ ]:
import pandas as pd
from datetime import datetime, time
import numpy as np

ES_df = pd.read_csv('./data/ES_5Years_8_11_2024.csv')
NQ_df = pd.read_csv('./data/NQ_5Years_8_11_2024.csv')
ES_df['datetime'] = pd.to_datetime(ES_df['Time'])
NQ_df['datetime'] = pd.to_datetime(NQ_df['Time'])

ES_df['Date'] = ES_df['datetime'].dt.date
ES_df['Time'] = ES_df['datetime'].dt.time
NQ_df['Date'] = NQ_df['datetime'].dt.date
NQ_df['Time'] = NQ_df['datetime'].dt.time

ES_df['datetime'] = ES_df['datetime'].dt.round('min')
NQ_df['datetime'] = NQ_df['datetime'].dt.round('min')
ES_df = ES_df.set_index('datetime')
NQ_df = NQ_df.set_index('datetime')

display(ES_df)
print(ES_df.dtypes)

In [ ]:
def generate_trade_data(df, dates, time_start, time_end):
    rows = []

    for date_str in dates:
        date_obj = pd.to_datetime(date_str).date()
        start_dt = pd.Timestamp(datetime.combine(date_obj, time_start)).round('min')
        end_dt = pd.Timestamp(datetime.combine(date_obj, time_end)).round('min')

        if start_dt in df.index and end_dt in df.index:
            start_price = df.loc[start_dt]['Close']
            end_price = df.loc[end_dt]['Close']
            diff = end_price - start_price

            rows.append({
                'date': date_str,
                'start_dt': start_dt,
                'end_dt': end_dt,
                'start_price': start_price,
                'end_price': end_price,
                'diff': diff
            })

    return pd.DataFrame(rows)

def backtest(trade_df, position='long'):
    if position == 'long':
        trade_df['PnL'] = trade_df['diff']
    elif position == 'short':
        trade_df['PnL'] = -trade_df['diff']
    else:
        raise ValueError("Position must be 'long' or 'short'")

    trade_df['cumulative_PnL'] = trade_df['PnL'].cumsum()
    return trade_df

dates = ['2023-11-30', '2024-02-29', '2024-05-31']
time_start = time(15, 30)
time_end = time(15, 55)

trade_df = generate_trade_data(ES_df, dates, time_start, time_end)
bt_result = backtest(trade_df, position='long')
display(bt_result)

In [ ]:
import matplotlib.pyplot as plt

def plot_pnl(trade_df):
    plt.figure(figsize=(10, 5))
    plt.plot(trade_df['date'], trade_df['cumulative_PnL'], marker='o')
    plt.title('Cumulative PnL')
    plt.xlabel('Date')
    plt.ylabel('PnL')
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

plot_pnl(bt_result)

In [ ]:
def calculate_annualized_sharpe(trade_df):
    start = pd.to_datetime(trade_df['date'].min())
    end = pd.to_datetime(trade_df['date'].max())
    num_business_days = np.busday_count(start.date(), end.date()) or 1  # prevent divide-by-zero

    mean_pnl = trade_df['PnL'].mean()
    std_pnl = trade_df['PnL'].std(ddof=0)

    if std_pnl == 0:
        sharpe = np.nan
    else:
        sharpe = (mean_pnl / std_pnl) * np.sqrt(252 / num_business_days)

    print(f"Annualized Sharpe Ratio: {sharpe:.4f}")
    return sharpe

calculate_annualized_sharpe(bt_result)